# Movie Script

#### Short discription of what happens in this script: 

Accessing and use the "Get Details" and "Get Reviews" endpoints of "The Movie Database API" (https://developers.themoviedb.org/3/getting-started/introduction) to obtain the desired data:
* the movie ID (that was requested)
* the corresponding movie title
* the corresponding movie reviews

Writing this data into Kafka via the Kafka "movieProducer" & reading the requested movie ID via the Kafka "movieConsumer". (The movie ID is necessary to determine for which movie the reviews should be retrieved.)

#### The individual steps:

Install Kafka library:

In [ ]:
#!pip install kafka-python

Import necessary packages:

In [1]:
import requests
# from Hidden_Secret import myApiKey
import re
import json
from Kafka_Helpers import Producer, Consumer



def get_data(movie_id):
    exampleID = movie_id
    # apiKey = myApiKey["apiKey"]
    apiKey = "105864a59e519ef281a74ca3af6c1b17"

    request1 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}?api_key=105864a59e519ef281a74ca3af6c1b17&language=en-US")
    response1 = request1.json()

    request2 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}/reviews?&language=en-US&page=1", {"api_key": apiKey})
    response2 = request2.json()
    result = response2['results']

    # ID
    movieID = [response1['id']]

    # original_title
    movieTitle = [response1['original_title']]

    # content 
    movieReviews = [item['content'] for item in result]
    movieReviewsSplitted = [re.sub(r"[^\w \- \  ]", "", item.lower()).split(" ") for item in movieReviews] # ToDo: How to ignore empty string?
    return movieID, movieTitle, movieReviewsSplitted

movieProducer = Producer('localhost', 29092)

def my_test_handler(key, value):
    movie_id, movie_title, movie_reviews = get_data(value)
    movieProducer.send("movie_reviews", "key", json.dumps({
        "movie_id": movie_id, 
        "title": movie_title, 
        "reviews": movie_reviews
    }))

movieConsumer = Consumer('localhost', 29092, "new_movie_id", my_test_handler)

Waiting for new events...


New event received


Process data:

Open Producer & write data into Kafka:

In [ ]:
# movieProducer = Producer('localhost', 29092)

# movieProducer.send("topic_b", "key", json.dumps({
#     "movieID": exampleID, 
#     "movieReviewsSplitted": movieReviewsSplitted, 
#     "movieTitle": movieTitle
# }))

Open Consumer & read data from Kafka:

In [ ]:
exampleID = 76601
# apiKey = myApiKey["apiKey"]
apiKey = "105864a59e519ef281a74ca3af6c1b17"

request1 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}?api_key=105864a59e519ef281a74ca3af6c1b17&language=en-US")
print(request1)
response1 = request1.json()

request2 = requests.get(f"https://api.themoviedb.org/3/movie/{exampleID}/reviews?&language=en-US&page=1", {"api_key": apiKey})
response2 = request2.json()
result = response2['results']

# ID
movieID = [response1['id']]

# original_title
movieTitle = [response1['original_title']]

# content 
movieReviews = [item['content'] for item in result]
movieReviewsSplitted = [re.sub(r"[^\w \- \  ]", "", item.lower()).split(" ") for item in movieReviews] # ToDo: How to ignore empty string?
movieID, movieTitle, movieReviewsSplitted